In [2]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import plotly.graph_objects as go

class CryptoCompareAPI:
    def __init__(self, api_key):
        self.api_key = api_key
        self.headers = {'authorization': f'Apikey {api_key}'}
        self.base_url = "https://min-api.cryptocompare.com/data"

    def get_top_list(self, limit=100):
        """Get current top coins"""
        url = f"{self.base_url}/top/mktcapfull"
        params = {
            'limit': limit,
            'tsym': 'USD'
        }
        
        try:
            response = requests.get(url, headers=self.headers, params=params)
            response.raise_for_status()
            return response.json()['Data']
        except Exception as e:
            print(f"Error fetching top list: {e}")
            return None

    def get_historical_daily_mcap(self, symbol, from_date, to_date=None):
        """Get historical market cap data for a coin"""
        url = f"{self.base_url}/v2/histoday"
        
        if to_date is None:
            to_date = datetime.now()
            
        from_timestamp = int(datetime.strptime(from_date, '%Y-%m-%d').timestamp())
        to_timestamp = int(to_date.timestamp())
        
        params = {
            'fsym': symbol,
            'tsym': 'USD',
            'limit': 2000,
            'toTs': to_timestamp
        }
        
        try:
            response = requests.get(url, headers=self.headers, params=params)
            response.raise_for_status()
            data = response.json()['Data']['Data']
            return {datetime.fromtimestamp(d['time']).strftime('%Y-%m-%d'): d['market_cap'] 
                   for d in data if d['time'] >= from_timestamp}
        except Exception as e:
            print(f"Error fetching historical data for {symbol}: {e}")
            return {}

def create_snapshots_and_track(api, snapshot_dates):
    """Create snapshots and track their market caps"""
    snapshots = {}
    market_cap_history = {}
    
    # Get current date for tracking until now
    today = datetime.now().strftime('%Y-%m-%d')
    
    # Create snapshots and track market caps
    for date in snapshot_dates:
        print(f"\nProcessing snapshot for {date}")
        
        # Get top 100 assets
        assets = api.get_top_list(limit=100)
        time.sleep(0.5)  # Rate limit compliance
        
        if not assets:
            continue
            
        # Store snapshot
        snapshot_label = f"{date}_t100"
        snapshots[snapshot_label] = []
        market_cap_history[snapshot_label] = {}
        
        # Process each asset
        for rank, asset in enumerate(assets, 1):
            symbol = asset['CoinInfo']['Name']
            snapshots[snapshot_label].append({
                'rank': rank,
                'symbol': symbol
            })
            
            # Get historical market cap data
            print(f"Getting historical data for {symbol}")
            hist_data = api.get_historical_daily_mcap(symbol, date)
            time.sleep(0.5)  # Rate limit compliance
            
            # Add to daily totals
            for day, mcap in hist_data.items():
                if day not in market_cap_history[snapshot_label]:
                    market_cap_history[snapshot_label][day] = 0
                market_cap_history[snapshot_label][day] += mcap
    
    return snapshots, market_cap_history

def create_visualization(market_cap_history):
    """Create visualization of aggregated market caps"""
    fig = go.Figure()
    
    for snapshot_label, history in market_cap_history.items():
        # Convert to dataframe for easier plotting
        df = pd.DataFrame(
            {'date': list(history.keys()),
             'market_cap': list(history.values())}
        ).sort_values('date')
        
        # Add trace
        fig.add_trace(go.Scatter(
            x=df['date'],
            y=df['market_cap'] / 1e9,  # Convert to billions
            name=snapshot_label,
            mode='lines',
            hovertemplate="Date: %{x}<br>" +
                         "Total Market Cap: $%{y:.2f}B<br>" +
                         "<extra></extra>"
        ))
    
    fig.update_layout(
        title='Aggregate Market Cap of Top 100 Assets from 1 and 2 Years Ago',
        xaxis_title='Date',
        yaxis_title='Total Market Cap (Billions USD)',
        hovermode='x unified',
        template='plotly_white',
        height=800,
        showlegend=True,
        legend_title='Snapshot Date'
    )
    
    return fig

# Initialize API and run
API_KEY = "0cde27853148238124877a771ce0b639b0e6528f9a878502368609849538ed2d"
api = CryptoCompareAPI(API_KEY)

# Define snapshot dates
dates = [
    "2023-03-13",  # 1 year ago
    "2022-03-13"   # 2 years ago
]

# Create snapshots and track market caps
print("Creating snapshots and tracking market caps...")
snapshots, market_cap_history = create_snapshots_and_track(api, dates)

# Print snapshots
print("\nSnapshots of Top 100 Assets:")
for date_label, assets in snapshots.items():
    print(f"\n{date_label}:")
    for asset in assets:
        print(f"Rank {asset['rank']}: {asset['symbol']}")

# Create and display visualization
print("\nCreating visualization...")
fig = create_visualization(market_cap_history)
fig.show()

# Save data
snapshot_df = pd.DataFrame([
    {'date_label': date_label, 'rank': asset['rank'], 'symbol': asset['symbol']}
    for date_label, assets in snapshots.items()
    for asset in assets
])
snapshot_df.to_csv('top_100_snapshots.csv', index=False)

mcap_df = pd.DataFrame([
    {'snapshot_date': snapshot_label, 'date': date, 'total_market_cap': mcap}
    for snapshot_label, history in market_cap_history.items()
    for date, mcap in history.items()
])
mcap_df.to_csv('market_cap_history.csv', index=False)

# Print some basic statistics
print("\nBasic Statistics:")
for snapshot_label, history in market_cap_history.items():
    mcaps = list(history.values())
    print(f"\n{snapshot_label}:")
    print(f"Initial Market Cap: ${min(mcaps)/1e9:.2f}B")
    print(f"Final Market Cap: ${max(mcaps)/1e9:.2f}B")
    print(f"Growth: {((max(mcaps)/min(mcaps))-1)*100:.2f}%")

Creating snapshots and tracking market caps...

Processing snapshot for 2023-03-13
Getting historical data for BTC
Error fetching historical data for BTC: 'market_cap'
Getting historical data for ETH
Error fetching historical data for ETH: 'market_cap'
Getting historical data for USDT
Error fetching historical data for USDT: 'market_cap'
Getting historical data for SOL
Error fetching historical data for SOL: 'market_cap'
Getting historical data for BNB
Error fetching historical data for BNB: 'market_cap'
Getting historical data for XRP
Error fetching historical data for XRP: 'market_cap'
Getting historical data for DOGE
Error fetching historical data for DOGE: 'market_cap'
Getting historical data for USDC
Error fetching historical data for USDC: 'market_cap'
Getting historical data for SUI
Error fetching historical data for SUI: 'market_cap'
Getting historical data for STETH
Error fetching historical data for STETH: 'market_cap'
Getting historical data for TON
Error fetching historical


Basic Statistics:

2023-03-13_t100:


ValueError: min() arg is an empty sequence